In [ ]:
import pandas as pd

In [ ]:
airports_df = pd.read_csv('../data/Airports.csv', encoding='ISO-8859-1')
airports_df = airports_df.tail(-1)
preview = airports_df.head()
preview

### Data

- TC_ID: Unique ID for the airport
- IATA: IATA airport code
- ICAO_OACI: ICAO airport code
- AIRPORT_T (and AÉROPORT_T): Type of airport in English (and French)
- TYPE_CODE: Numeric code indicating the type of airport
- AIRPORT_AÉROPORT: Name of the airport
- CITY_VILLE: City where the airport is located
- PROVINCE_EN (and PROVINCE_FR): Province in English (and French)
- LATTITUDE: Latitude of the airport
- LONGITUDE: Longitude of the airport

### Let's improve the data storage
1. Instead of having all airports as a flat structure, we can nest them under their respective provinces. 
2. We can categorize airports by their type and provide a tag-based structure.
3. Include a geohash string of the latitude and longitude airports.



In [ ]:
cleaned_airports_df = airports_df.drop(columns=['ID', 'TYPE_CODE', 'AROPORT_T', 'PROVINCE_FR', 'AIRPORT_AROPORT'])
cleaned_airports_df = cleaned_airports_df.rename(columns={
    'CODE': 'airport_code',
    'C_CODE': 'icao_code',
    'TYPE': 'airport_type',
    'CITY': 'city',
    'PROV': 'province',
    'LAT': 'latitude',
    'LONG': 'longitude'
})

cleaned_airports_df.head()

In [ ]:
provincial_data = cleaned_airports_df.groupby('province').apply(lambda x: x.drop('province', axis=1).to_dict(orient='records')).to_dict()

provincial_data_example = {key: provincial_data[key] for key in list(provincial_data.keys())[:1]}
provincial_data_example


In [ ]:
def compute_geohash(latitude, longitude, precision=6):
    """
    Compute a geohash for given latitude and longitude.
    """
    lat_interval, lon_interval = [-90.0, 90.0], [-180.0, 180.0]
    base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
    geohash = []
    bits = [16, 8, 4, 2, 1]
    bit = 0
    ch = 0
    even = True
    while len(geohash) < precision:
        if even:
            mid = (lon_interval[0] + lon_interval[1]) / 2
            if float(longitude) > mid:
                ch |= bits[bit]
                lon_interval = (mid, lon_interval[1])
            else:
                lon_interval = (lon_interval[0], mid)
        else:
            mid = (lat_interval[0] + lat_interval[1]) / 2
            if float(latitude) > mid:
                ch |= bits[bit]
                lat_interval = (mid, lat_interval[1])
            else:
                lat_interval = (lat_interval[0], mid)
        even = not even
        if bit < 4:
            bit += 1
        else:
            geohash.append(base32[ch])
            bit = 0
            ch = 0
    return ''.join(geohash)

cleaned_airports_df['geohash'] = cleaned_airports_df.apply(lambda x: compute_geohash(x['latitude'], x['longitude']), axis=1)

cleaned_airports_df.loc[:, ['airport_code', 'city', 'latitude', 'longitude', 'geohash']].head()


In [47]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two points on the earth in kilometers.
    """
    R = 6371.0  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = float(lat1), float(lon1), float(lat2), float(lon2)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def get_three_closest_airports(row):
    distances = cleaned_airports_df.apply(
        lambda x: haversine_distance(row['latitude'], row['longitude'], x['latitude'], x['longitude']), 
        axis=1
    )

    closest_three = distances.nsmallest(4).index[1:4] - 1
    return cleaned_airports_df.iloc[closest_three]['airport_code'].tolist()

cleaned_airports_df['three_closest_airports'] = cleaned_airports_df.apply(get_three_closest_airports, axis=1)

cleaned_airports_df.head()


,airport_code,icao_code,airport_type,city,province,latitude,longitude,geohash,three_closest_airports
1,YXY,CYXY,Control Tower,Whitehorse,Yukon,60.716666,-135.066666,bfzcqm,"[YVQ, YYE, YPR]"
2,YYJ,CYYJ,Control Tower,Victoria,British Columbia,48.65,-123.433333,c28hz0,"[YWH, YCD, YDT]"
3,YYT,CYYT,Control Tower,Saint John,Newfoundland and Labrador,47.616666,-52.75,fb6qzs,"[YQX, YDF, YGR]"
4,YQX,CYQX,Control Tower,Gander,Newfoundland and Labrador,48.933333,-54.566666,fb9nqk,"[YYT, YDF, YGR]"
5,YHZ,CYHZ,Control Tower,Halifax,Nova Scotia,44.883333,-63.516666,dxfz6d,"[YYG, YQM, YSJ]"


In [48]:
airports_json = cleaned_airports_df.to_json(orient="records")

json_file_path = "../data/airports_processed.json"
with open(json_file_path, 'w') as f:
    f.write(airports_json)

json_file_path


'../data/airports_processed.json'